<a href="https://colab.research.google.com/github/project-anya/MachineLearning/blob/main/Ibu_Food_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd

/content


In [ ]:
!ls

dataset-makanan-ibu.zip  sample_data


In [ ]:
!unzip /content/dataset-makanan-ibu -d /content/Images

Archive:  /content/dataset-makanan-ibu.zip
   creating: /content/Images/dataset-makanan-ibu/
   creating: /content/Images/dataset-makanan-ibu/Kalio Ayam/
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/d194339b25.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/8211e08f37.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/474210b1bb.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/3db1ad4ba1.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/6087bab584.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/5dfea9b405.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/43790dd35e.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/f5c8d22f5c.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/89311b7a30.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/ce7b013ce7.jpg  
  inflating: /content/Images/dataset-makanan-ibu/Kalio Ayam/fd4e3cb12d.jpg  

In [ ]:
import os

dataset_path = '/content/Images/dataset-makanan-ibu'
list_class = os.listdir(dataset_path)
list_class

['Beef Burger',
 'Ayam Taliwang',
 'Nasi Rames',
 'Bubur Ayam',
 'Martabak Mesir',
 'Ketoprak',
 'Mie Pangsit Basah',
 'Kalio Ayam',
 'Sop Daging Sapi',
 'Rendang Sapi',
 'Mie Aceh Rebus',
 'Mie Ayam',
 'Karedok',
 'Chicken Teriyaki',
 'Beef Teriyaki',
 'Soto Betawi',
 'Gado-Gado',
 'Soto Padang',
 'Mie Bakso',
 'Pempek Telur']

In [ ]:
import tensorflow as tf
import cv2 as cv 
from matplotlib import pyplot as plt 

In [ ]:
count_class = [] 
for name in list_class : 
  path = os.path.join(dataset_path, name)
  count_class.append(len(os.listdir(path)))

In [ ]:
for i in range (len(count_class)) :
  print (list_class[i], count_class[i])

Beef Burger 149
Ayam Taliwang 162
Nasi Rames 159
Bubur Ayam 169
Martabak Mesir 157
Ketoprak 166
Mie Pangsit Basah 139
Kalio Ayam 172
Sop Daging Sapi 178
Rendang Sapi 172
Mie Aceh Rebus 159
Mie Ayam 179
Karedok 175
Chicken Teriyaki 165
Beef Teriyaki 171
Soto Betawi 174
Gado-Gado 165
Soto Padang 146
Mie Bakso 157
Pempek Telur 172


In [21]:
images = []
classes = []

In [22]:
for idx, name in enumerate(list_class) : 
  path = os.path.join(dataset_path, name)
  dir = os.listdir(path)
  for image in dir : 
    img_path = os.path.join(path, image)
    img_gray = cv.imread(img_path, 0)
    images.append(img_gray)
    classes.append(idx)

print(len(images))

3286


In [24]:
from sklearn.model_selection import train_test_split

input_train, input_test, output_train, output_test = train_test_split(images, classes, test_size=0.3, random_state=69)

input_train, input_valid, output_train, output_valid = train_test_split(input_train, output_train, test_size=0.2, random_state=23)

In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
    )
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [28]:
image_size = (200, 200)

In [30]:
from sklearn.utils import shuffle
training = train_datagen.flow_from_directory(dataset_path, 
                                             target_size = image_size,
                                             batch_size = 32,
                                             class_mode='categorical',
                                             shuffle=True,
                                             subset='training'
                                             )
valid = valid_datagen.flow_from_directory(dataset_path,
                                          target_size=image_size,
                                          batch_size = 32,
                                          shuffle=True,
                                          subset='validation',
                                          class_mode='categorical'
                                          )

Found 2638 images belonging to 20 classes.
Found 648 images belonging to 20 classes.


In [49]:
base_model = tf.keras.applications.vgg19.VGG19(input_shape=(200,200,3),include_top=False, weights='imagenet')
base_model.trainable = False

print(base_model.summary())

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [50]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import Model


In [58]:
x = base_model.output

# x = layers.Dropout(0.2)(x)
x = layers.Conv2D(128, (3,3), activation='relu')(x)
# x = layers.Conv2D(2048, (3,3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
# x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
# x = layers.Dense(512, activation='relu')(x)
# x = layers.Dropout(0.2)(x)
x = layers.Dense(20, activation='softmax')(x)

models = Model(inputs=base_model.input, outputs= x)
print(models.summary())

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0   

In [ ]:
models.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = models.fit(training, epochs=30, steps_per_epoch=32, validation_data=valid, verbose=1, validation_steps=6)

Epoch 1/30
32/32 [==============================] - 646s 20s/step - loss: 2.9033 - accuracy: 0.1143 - val_loss: 2.5444 - val_accuracy: 0.2292
Epoch 2/30
32/32 [==============================] - 640s 20s/step - loss: 2.5385 - accuracy: 0.2028 - val_loss: 2.4648 - val_accuracy: 0.2135
Epoch 3/30
32/32 [==============================] - 631s 20s/step - loss: 2.3869 - accuracy: 0.2406 - val_loss: 2.2639 - val_accuracy: 0.2969
Epoch 4/30
17/32 [==============>...............] - ETA: 4:11 - loss: 2.2398 - accuracy: 0.3070